In [29]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Cleaned Datasets/premier_league_features_df.csv'
path_2 = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Cleaned Datasets/combined_elo_with_features_df.csv'


premier_league_features_df = pd.read_csv(path)
combined_elo_with_features_df = pd.read_csv(path_2)

missing_information_list= ['segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2012.0',
 'segunda_liga: 2012.0']

Missing home ELO Values = ['championship', 'premier_league', 'eredivisie', 'primeira_liga',
       'ligue_2', 'segunda_division', 'primera_division', 'serie_b',
       'serie_a', '2_liga']

In [30]:
# Function to remove partially completed seasons

def _remove_incomplete_seasons(missing_information_list, combined_elo_with_features_df):

    missing_info_set = set(missing_information_list)
    missing_info_deduplicated_list = list(missing_info_set)
    missing_info_deduplicated_list
    for item in missing_info_deduplicated_list:
        league = item.split(": ")[0]
        season = item.split(": ")[1]
        combined_elo_with_features_df.drop(combined_elo_with_features_df.index[(combined_elo_with_features_df.League == league)&(
                        combined_elo_with_features_df.Season == season)], inplace=True)
    
    return combined_elo_with_features_df

def _prepare_dataset_for_ML_models(df):


    df.drop(axis=1, columns=['Result','Round', 'Link', 'Home_Win', 'Away_Win', 'Date_New', 'Home_Team', 'Away_Team', 'Season', 'League', 'Referee', 'City', 'Country', 'Stadium', 'Pitch','Home_Yellow', 'Home_Red', 'Away_Yellow', 'Away_Red'], inplace=True)
    df["Goal_Difference"] = df["Home_Goals"] - df["Away_Goals"]
    df.drop(axis=1, columns=['Home_Goals', 'Away_Goals'], inplace=True)
    df.dropna(subset=["ELO_home"], inplace = True)
    
    return df



In [31]:
combined_elo_with_partial_seasons_removed_df = _remove_incomplete_seasons(missing_information_list, combined_elo_with_features_df)
combined_elo_numerical_df = _prepare_dataset_for_ML_models(combined_elo_with_partial_seasons_removed_df)

combined_elo_numerical_df[combined_elo_numerical_df["ELO_home"].isnull()]

,Capacity,ELO_home,ELO_away,home_team_total_goals_scored_so_far,home_team_total_goals_conceeded_so_far,home_team_current_win_streak,home_team_current_loss_streak,home_team_total_points_so_far,home_team_current_goal_drought,home_team_total_wins_so_far,away_team_total_goals_scored_so_far,away_team_total_goals_conceeded_so_far,away_team_current_win_streak,away_team_current_loss_streak,away_team_total_points_so_far,away_team_current_goal_drought,away_team_total_wins_so_far,Goal_Difference


In [33]:
from decouple import Config, RepositoryEnv
from sqlalchemy import create_engine


def _connect_to_RDS() -> create_engine:
    '''_connect_to_RDS 
    Method to create connection to RDS database using sqlalchemy

    Returns:
        engine - connection

    '''
    DOTENV_FILE = '/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /match-outcome-predictor/.env'
    env_config = Config(RepositoryEnv(DOTENV_FILE))

    RDS_DATABASE_PASSWORD = env_config.get('RDS_DATABASE_PASSWORD')
    RDS_DATABASE_TYPE = env_config.get('RDS_DATABASE_TYPE')
    RDS_DBAPI = env_config.get('RDS_DBAPI')
    RDS_ENDPOINT = env_config.get('RDS_ENDPOINT')
    RDS_USER = env_config.get('RDS_USER')
    RDS_DATABASE = env_config.get('RDS_DATABASE')
    PORT = 5432


    engine = create_engine(
            f"{RDS_DATABASE_TYPE}+{RDS_DBAPI}://{RDS_USER}:{RDS_DATABASE_PASSWORD}@{RDS_ENDPOINT}:{PORT}/{RDS_DATABASE}")
        
    return engine

def _upload_dataframes_to_rds(df):
    '''_upload_dataframes_to_rds 
    If dataframe exists, append it to SQL database.
    '''

    engine = _connect_to_RDS()
    df.to_sql('match_outcomes', engine, if_exists='append')

   



In [35]:
_upload_dataframes_to_rds(combined_elo_numerical_df)

In [36]:
path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Cleaned Datasets/cleaned_dataset.csv'
combined_elo_numerical_df.to_csv(path, index=False)